**폰트 설치**

In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 42 not upgraded.
Need to get 9,604 kB of archives.
After this operation, 29.5 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 fonts-nanum all 20170925-1 [9,604 kB]
Fetched 9,604 kB in 2s (5,284 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletyp

**필요한 라이브러리 import**

### 라이브러리 설명

itertools library : 효율적인 looping을 위한 iterator를 만드는 라이브러리


* Repeat

 list(repeat(object, n)): object를 n번 반복해서 하나로 묶어 줍니다

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import itertools
from itertools import repeat

**데이터 불러오기**

2. 영업시간,배달여부 에서 사용하는 데이터

 -소상공인시장진흥공단 상가(상권)정보.csv
 
 -크롤링 폴더에 위치한 서대문구 각 행정동명의 영업시간,배달 crawling data

In [ ]:
#데이터를 불러오기 위한 드라이브 마운트
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
df = pd.read_csv('/content/gdrive/MyDrive/파데분플젝/크롤링/shops.csv', sep=',')  #소상공인시장진흥공단 상가(상권)정보.csv를 shops.csv로 불러옵니다
columns = ['상호명', '상권업종대분류명','상권업종소분류명', 
           '시도명', '시군구명', '행정동명', '도로명주소', 
           '경도', '위도']
cond = df['시군구명'] == '서대문구' #본 프로그램은 서대문구의 경우만 이용합니다
df = df[cond]
dong = sorted(set(list((df['행정동명'])))) #서대문구의 동 목록을 정렬하여 가져옵니다

In [ ]:
print('서대문구의 행정동명은 다음과 같습니다 : \n', dong)

print()
region = input('행정동명을 입력해주세요: ') #서대문구의 동 중 사용자가 원하는 행정동명을 입력합니다

서대문구의 행정동명은 다음과 같습니다 : 
 ['남가좌1동', '남가좌2동', '북가좌1동', '북가좌2동', '북아현동', '신촌동', '연희동', '천연동', '충현동', '홍은1동', '홍은2동', '홍제1동', '홍제2동', '홍제3동']

행정동명을 입력해주세요: 남가좌2동


In [ ]:
#입력한 행정동에 해당하는 '배달_영업시간.csv' 파일을 dataframe으로 불러옵니다
df = pd.read_csv('/content/gdrive/MyDrive/파데분플젝/크롤링/영업시간,배달 crawling data/'+region+'_result.csv')

### **메인 프로그램**
matplotlib.pyplot 라이브러리를 활용해 영업 시간을 시각화해주고, 배달 가능 여부를 알려줍니다.

가게의 영업 시간과 배달 여부를 사용자가 원하는 하에 연속적으로 확인할 수 있습니다.

In [ ]:
#while무한루프 안에서 사용자가 프로그램 종료를 입력할 때까지 영업 시간과 배달 시간 정보를 알 수 있습니다
while True:
    mode = int(input('모드를 골라주세요: 1 - 검색 모드: , 2 - 프로그램 종료: '))


    if mode == 1: #검색 모드의 경우, 음식점을 입력해 원하는 음식점의 영업 시간과 배달 여부를 알 수 있습니다
        print()
        store=input("영업시간과 배달 가능 여부를 알고 싶은 가게 이름을 입력하세요: ") #음식점의 이름을 입력받습니다
        print()
        ii=df.index[df['상호명'].str.contains(store)].tolist() #가게의 이름을 다 기억하지 못해 일부만 입력하더라도 작동하도록 합니다
        i=ii[0] #index로 사용할 수 있도록  i를 list의 요소로 가져옵니다
        start_arr=str(df['office_hour_start'][i]) #영업 시작 시간을 문자열 형태로 저장합니다
        end_arr=str(df['office_hour_end'][i]) #영업 종료 시간을 문자열 형태로 저장합니다
        if ':' in start_arr and ':' in end_arr: #네이버 플레이스에 영업 시작 시간과 영업 종료 시간이 모두 정상적으로 표시(00:00 형태)됐을 경우 영업 시간을 시각화합니다
            start=int(df['office_hour_start'][i][:2]) #영업 시작 시간의 '시:분' 중 '시' 부분
            end=int(df['office_hour_end'][i][:2]) #영업 종료 시간의 '시:분' 중 '시' 부분
            x=np.arange(start,end+1) #영업 시간이 몇시부터 몇시인지 np.arange()로 담아둡니다
            y=list(repeat(1,len(x))) #영업 시간에 해당하는 시간은 1을 값으로 가지도록
            df_=pd.DataFrame(y,x,columns=['영업 중'])
            df_.index = df_.index.map(int)
            plt.style.use('ggplot')
            plt.rc('font', family='NanumBarunGothic')
            #x,y로 만든 dataframe에 plot을 적용합니다
            df_.plot(kind='area',stacked=False,alpha=0.2,figsize=(14,0.7),xticks=np.arange(0,25),yticks=np.arange(2,3)) #kind='area'를 통해 영업시간에 해당되면 해당 영역에 색으로 표시를 합니다
            plt.title('<'+df['상호명'][i]+'>'+"의 영업시간: "+df['office_hour_start'][i]+'~'+df['office_hour_end'][i]) 
            plt.xlabel('00:00a.m.~24:00p.m.',size=10)
            plt.ylabel('영업',size=10)
            plt.show()
        else: #네이버 플레이스에 영업시간이 정상적으로 표시되지 않은 경우('휴무' 표시 또는 표시되어 있지 않음 등) 표시되지 않았다는 문구를 보여줍니다
            print()
            print('❗'+'<'+df['상호명'][i]+">은 영업시간이 표시되어 있지 않은 가게입니다"+'❗')
        print()
        print('   배달 가능 여부')
        if df['delivery_bool'][i]=='Yes': #df에 저장해 놨던 배달 가능 여부가 yes일 경우 배달 가능이라고 표시
            print('🛵'+'<'+df['상호명'][i]+">은 배달 가능한 가게입니다")
            print('______________________________________________________________________________________________________________________')
        else:
            print('❗'+'<'+df['상호명'][i]+">은 배달이 불가능하거나 배달 여부가 표시되어 있지 않은 가게입니다"+'❗')
            print('______________________________________________________________________________________________________________________')
        
    elif mode == 2: #프로그램 종료를 선택할 경우 break를 통해 반복문을 종료합니다
        print()
        print('이용해 주셔서 감사합니다')
        break    
    
    else: #잘못된 숫자를 입력했을 경우 고지한 다음 반복문을 끝내지 않고 다시 입력을 받습니다
        print()
        print('잘못된 숫자를 입력하셨습니다. 다시 입력해주세요 ')
        print()

모드를 골라주세요: 1 - 검색 모드: , 2 - 프로그램 종료: 1

영업시간과 배달 가능 여부를 알고 싶은 가게 이름을 입력하세요: 봄날


❗<봄날>은 영업시간이 표시되어 있지 않은 가게입니다❗

   배달 가능 여부
❗<봄날>은 배달이 불가능하거나 배달 여부가 표시되어 있지 않은 가게입니다❗
______________________________________________________________________________________________________________________
모드를 골라주세요: 1 - 검색 모드: , 2 - 프로그램 종료: 2

이용해 주셔서 감사합니다
